# Pyntcloud Visualizer

The following notebook is an example of using [pyntcloud](https://github.com/daavoo/pyntcloud).

Most of pyntcloud's functionallity can be accesed through the core class: [PyntCloud](http://pyntcloud.readthedocs.io/en/latest/PyntCloud.html).

In [1]:
from pyntcloud import PyntCloud

ModuleNotFoundError: No module named 'pyntcloud'

And this is the only import we need for now.

## Loading 3D data

The first thing we are going to do is load the 3D data into Python, using PyntCloud's class method: [from_file](http://pyntcloud.readthedocs.io/en/latest/io.html)

In [2]:
scene = PyntCloud.from_file("scene3.ply")
scene2 = PyntCloud.from_file("scene3.ply") #Two were used to save the floor in one separate file

In [3]:
scene # Factory

PyntCloud
338199 points with 0 scalar fields
671203 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: -1276.465576171875, 2136.514404296875, 1098.029296875
Other attributes:

In [4]:
a = (scene.mesh)
scene.points = scene.points/200 #Hardcoded scalining because of the distance between points in CAD
scene.mesh = a
a = (scene2.mesh)
scene2.points = scene2.points/200
scene2.mesh = a

In [5]:
scene.points.head()

x          y    z
0  -9.179413  17.206535  0.0
1 -13.949665   5.280414  0.0
2  -9.174702  17.191383  0.0
3 -18.699665  24.847401  0.0
4  -8.707599  17.847338  0.0

## 3D point cloud from 3D mesh

Luckely for us, we can convert a triangular mesh into a point cloud:

In [6]:
# sample 500000 points from the mesh
sampled_points = scene.get_sample("mesh_random_sampling",
                                  n=500000)
sampled_points2 = scene2.get_sample("mesh_random_sampling",
                                  n=500000)

# manually construcst a new PyntCloud with those points
scene = PyntCloud(sampled_points)
scene2 = PyntCloud(sampled_points2)

We can verify the result using the `__repr__` method mentioned above (this time using the `print` function) ...

In [7]:
print(scene)

PyntCloud
500000 points with 0 scalar fields
0 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: -4.274939060211182, 13.179546356201172, 1.7366610765457153
Other attributes:



In [8]:
scene.plot()

## Floor segmentation

We are not interested in the points that compose the floor, so we will use [RANSAC](https://es.wikipedia.org/wiki/RANSAC) to find the plane that contains those points.

This is a good example of using the function [add_scalar_field](http://pyntcloud.readthedocs.io/en/latest/scalar_fields.html), wich will add a new column to `points` and return a `string` with the name of that column.

It's a good practice to store the strings returned `from PyntCloud.`**add_**`*` methods. As you will see later, this it quite convenient when chaining operations that require those strings as arguments.

In [10]:
is_floor = scene.add_scalar_field("plane_fit",
                                  n_inliers_to_stop=len(scene.points)/30)
is_floor2 = scene2.add_scalar_field("plane_fit",
                                  n_inliers_to_stop=len(scene2.points)/30)

This method adds a new column to points.

In [11]:
scene.points.head()

x          y         z  is_plane
0 -10.167369  12.329792  0.000000         1
1   5.821401  24.013952  0.000000         1
2   3.648916  13.028890  3.302000         0
3 -11.993650   7.762948  0.000000         1
4   3.165603  21.072624  5.468216         0

In [12]:
# creates a boolean array
not_floor = scene.points[is_floor] != 1 
floor = scene2.points[is_floor] == 1 

In [13]:
scene.apply_filter(not_floor)
scene2.apply_filter(floor)

In [14]:
scene.points.head()

x          y          z  is_plane
0   3.648916  13.028890   3.302000         0
1   3.165603  21.072624   5.468216         0
2 -10.100225  11.874265  10.432081         0
3   0.587231  16.770767   1.232295         0
4   3.397451  23.161665   2.202228         0

In [15]:
scene.plot(output_name="without_floor")

## Clustering robots

Let's separate the points that belong to each robot using a simple clustering technique called euclidean clustering. 

This is a good example of chaining `add_` operations storing the string indetifiers in variables.

First, we will add a VoxelGrid to the PyntCloud using [add_structure](http://pyntcloud.readthedocs.io/en/latest/structures.html).

In [16]:
vg_id = scene.add_structure("voxelgrid", sizes=[1,1,2])

Using the voxelgrid, we can add a new scalar field that indicates to wich cluster each point belong.

In [17]:
clusters_id = scene.add_scalar_field("euclidean_clusters", voxelgrid=vg_id)
scene.split_on(clusters_id, save_path="tmp")

In [18]:
scene.plot(use_as_color=clusters_id, cmap="viridis")

### Local files are generated in tmp, and sent to the network for labling manually
### The resulting labels are fed back into pyntcloud the plot to be visualized

In [19]:
#To hardcode new values
scene.points[is_floor] = scene.points[clusters_id]
#for i in range(len(scene.points[is_floor])):
    #if scene.points[is_floor][i] == 4.0:
        #scene.points[is_floor][i] = 3.0

In [20]:

scene2.points[is_floor] = 5.0

In [21]:
import pandas as pd
print(scene)
scene3 = [scene.points, scene2.points]
result = pd.concat(scene3)
scene.points = result

PyntCloud
247453 points with 2 scalar fields
0 faces in mesh
0 kdtrees
1 voxelgrids
Centroid: -3.2498676776885986, 13.474801063537598, 3.5090720653533936
Other attributes:



## Visualization of labels of objects in a scene

In [22]:
scene.plot(use_as_color=is_floor, cmap="viridis")